In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory

In [2]:

# Imports for using Ollama
from langchain_ollama import OllamaEmbeddings # Ollama Embeddings
from langchain_ollama import ChatOllama # Ollama LLM Model Holder

In [3]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
llm = ChatOllama(model="llama3.2")

In [4]:

db_location = "./chroma_langchain_db"

In [5]:
# Dividing Document into chunks and storing in vector database

if not os.path.exists(db_location):
    print("Database not found. Running initial setup...")
    loader = PyPDFLoader(r"D:\\Ragbot\\test\\Agni.pdf") # PDF File path
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
    chunks = text_splitter.split_documents(pages)
    vector_store = Chroma.from_documents(chunks, embeddings, persist_directory=db_location)
    print("Documents added to Chroma DB.")
    exit("Please create the Chroma database first by running the setup code.")

Database not found. Running initial setup...
Documents added to Chroma DB.


In [6]:
# Loads the initial database and initializes retriever
vector_store = Chroma(persist_directory=db_location, embedding_function=embeddings)
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [7]:
# Prompt 1: To rephrase the user's question based on history
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [8]:
# Prompt 2: To answer the question using the retrieved context
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


In [9]:
# Chain combined with history and qa chain

rag_chain_with_history = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [10]:
# Question Answer with History

chat_history = ChatMessageHistory()

def ask_with_history(query: str):
    global chat_history
    
    if query.lower() == "clear history":
        chat_history.clear()
        print("Chat history cleared. Ask me a new question.")
        return
    
    result = rag_chain_with_history.invoke({
        "input": query,
        "chat_history": chat_history.messages
    })
    print(result["answer"])
    
    chat_history.add_user_message(query)
    chat_history.add_ai_message(result["answer"])

In [11]:
# Example prompt to clear history
# ask_with_history("clear history")

In [12]:
# Example Question
ask_with_history("Who is Agni?")

Agni is the Vedic god of fire, revered as a divine presence that bridges the human and divine realms. He is also considered the principle of divine communication, transformation, and continuity, symbolizing energy, passion, and life force. Agni is not just a god of fire but an essential mediator of ritual.


In [13]:
# Example Question to demonstrate chatting with history
ask_with_history("What did I just asked?")

You asked "Who is Agni?" and I provided a response. However, I should have been more precise in my answer. You essentially asked "Who" or "What is" Agni, rather than asking about his characteristics or role. My initial response was an attempt to provide more context, but I can try again with a simpler answer if you'd like.


In [ ]:
# Example prompt to clear history
ask_with_history("clear history")


Chat history cleared. Ask me a new question.


In [ ]:
# Example Question to demonstrate chatting with history
# In this case after clearing the chat history
ask_with_history("Did I asked something related to Agni?")

No, you didn't ask anything about Agni. This conversation just started with the context provided about Agni.
